In [1]:
import gym
import numpy as np
import FerchNN
from random import sample

In [2]:
env = gym.make("MountainCarContinuous-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
env.action_space.high

array([ 1.], dtype=float32)

In [4]:
mean_nn = FerchNN.NN_Network(input_size = env.observation_space.shape[0], lr =0.000001, output_size = env.action_space.shape[0] ,  activation = "linear")
std_nn = FerchNN.NN_Network(input_size = env.observation_space.shape[0], lr =0.000001, output_size = env.action_space.shape[0] , activation = "linear")

In [5]:
timee = 0
def std_transformation(std, c = 500):
    global timee
    return np.power(std,2)+4*np.power(np.e, -(timee/c))

In [6]:
def choose_action(state):
    
    action = np.zeros(env.action_space.shape[0])
    mean = mean_nn.predict(state)
    std = std_nn.predict(state)
    #std_aux= np.log(1+np.power(np.e, std))+0.00001
    std_aux = std_transformation(std)
    #std_aux = np.zeros(env.action_space.shape)
    
    for i in range(env.action_space.shape[0]):
        
        action[i] = np.random.normal(mean[i], std_aux[i])
        
        if action[i]>= env.action_space.high[i]:
            action[i] = env.action_space.high[i]
        
        if action[i]<= env.action_space.low[i]:
            action[i] = env.action_space.low[i]
            
        
    
    #std = std_aux
    #print(mean, std_aux)
    return action, mean, std 

In [7]:
def compute_gradient(action, reward, mean, std):
    
    #std_aux= np.log(1+np.power(np.e, std))+0.00001
    std_aux = std_transformation(std)
    mean_gradient = -reward*((action - mean)/np.power(std_aux,2))
    #mean_gradient = (mean_gradient)/np.linalg.norm(mean_gradient)
     
    #std_gradient = -reward*((np.power((action-mean), 2)-np.power(std_aux,2))/np.power(std_aux,3))*( 1/ (1+np.power(np.e,-std)))
    std_gradient = -reward*((np.power((action-mean), 2)-np.power(std_aux,2))/np.power(std_aux,3))*2*std
    #std_gradient = (std_gradient)/ np.linalg.norm(std_gradient)
    
    
    return mean_gradient, std_gradient

# REINFORCE

In [8]:
def compute_cumulative_rewards(rewards, gamma = 0.99):
    
    r_aux = np.zeros(len(rewards))
    
    for i in range(len(rewards)):
        discounted_rewards = 0
        
        for j in range(i, len(rewards)): 
            discounted_rewards = discounted_rewards + np.power(gamma, j-i)*rewards[j]
        
        r_aux [i] = discounted_rewards
    
    return r_aux

In [ ]:
def reinforce(actions, means, stds, rewards):
    
    r = compute_cumulative_rewards(rewards)
    
    assert len(actions)== len(means)
    assert len(means) == len(stds)
    assert len(stds) == len(rewards)
    
    for i in range(len(actions)):
        mean_gradient, std_gradient  = compute_gradient(actions[i], r[i], means[i], stds[i])
        mean_nn.fit(None,None, dloss = mean_gradient)
        std_nn.fit(None, None, dloss = std_gradient)

In [ ]:
n_episodes = 100000
horizon = 1000
for i_episode in range(n_episodes):
    
    state = env.reset()
    global time
    timee = i_episode
    actions = []
    means = []
    stds = []
    rewards = []
    time = 0
    for t in range(horizon):
        time+=1
        env.render()
        ########################################
        action, mean, std = choose_action(state)
        actions.append(action)
        means.append(mean)
        stds.append(std)
        ########################################
        next_state, reward, done, info = env.step(action)
        rewards.append(reward)
        
        state = next_state
        if done:
            break
    reinforce(actions, means, stds, rewards)
    
    print("-------------------------------")
    print("Episode finished after {} timesteps".format(time+1))
    print(i_episode, " Reward ", np.array(rewards).sum())
    print(i_episode, " Mean ", np.array(means).mean())
    print(i_episode, " Std ", std_transformation(np.array(stds)).mean())
    
env.close()

D:\Fernando\Documents\TCC\continuous_action_space\FerchNN.py:210: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if dloss != None:


-------------------------------
Episode finished after 1000 timesteps
0  Reward  -99.8057024654
0  Mean  29.8496860894
0  Std  881.229867205
-------------------------------
Episode finished after 1000 timesteps
1  Reward  -99.9
1  Mean  29.853373261
1  Std  881.459293862
-------------------------------
Episode finished after 1000 timesteps
2  Reward  -99.8332720351
2  Mean  29.8533742482
2  Std  881.48714898
-------------------------------
Episode finished after 1000 timesteps
3  Reward  -99.8909618718
3  Mean  29.853374593
3  Std  881.514960445
-------------------------------
Episode finished after 1000 timesteps
4  Reward  -99.8091358342
4  Mean  29.8533749388
4  Std  881.542809671
-------------------------------
Episode finished after 1000 timesteps
5  Reward  -99.8987303559
5  Mean  29.8533752838
5  Std  881.570643035
-------------------------------
Episode finished after 1000 timesteps
6  Reward  -99.8505672393
6  Mean  29.8533756297
6  Std  881.598527698
-------------------------

-------------------------------
Episode finished after 1000 timesteps
57  Reward  -85.6959544982
57  Mean  -0.14881573261
57  Std  3.76221164189
-------------------------------
Episode finished after 907 timesteps
58  Reward  24.5756744298
58  Mean  -0.148872034125
58  Std  3.75657662175
-------------------------------
Episode finished after 1000 timesteps
59  Reward  -85.5761302081
59  Mean  -0.148821919046
59  Std  3.74881996047
-------------------------------
Episode finished after 1000 timesteps
60  Reward  -83.6598708673
60  Mean  -0.148878598505
60  Std  3.74321596429
-------------------------------
Episode finished after 1000 timesteps
61  Reward  -85.5491717614
61  Mean  -0.148936307061
61  Std  3.73761797311
-------------------------------
Episode finished after 1000 timesteps
62  Reward  -84.465518305
62  Mean  -0.149019506912
62  Std  3.73206413119
-------------------------------
Episode finished after 1000 timesteps
63  Reward  -85.4151216016
63  Mean  -0.149108173268
63  S

-------------------------------
Episode finished after 1000 timesteps
114  Reward  -83.8215963865
114  Mean  -0.152276381191
114  Std  3.47332153775
-------------------------------
Episode finished after 1000 timesteps
115  Reward  -83.4034475056
115  Mean  -0.152358716914
115  Std  3.46929231987
-------------------------------
Episode finished after 1000 timesteps
116  Reward  -86.3145998175
116  Mean  -0.152408916909
116  Std  3.46530640216
-------------------------------
Episode finished after 1000 timesteps
117  Reward  -85.4211029737
117  Mean  -0.152450535627
117  Std  3.46142297301
-------------------------------
Episode finished after 1000 timesteps
118  Reward  -85.7594749543
118  Mean  -0.152514545203
118  Std  3.45754082939
-------------------------------
Episode finished after 821 timesteps
119  Reward  30.5862153097
119  Mean  -0.152618565675
119  Std  3.45370708632
-------------------------------
Episode finished after 1000 timesteps
120  Reward  -84.9649854732
120  Mean 

-------------------------------
Episode finished after 1000 timesteps
170  Reward  -83.9584318865
170  Mean  -0.155655599717
170  Std  3.2788744801
-------------------------------
Episode finished after 1000 timesteps
171  Reward  -83.8474080884
171  Mean  -0.155689103117
171  Std  3.27687438389
-------------------------------
Episode finished after 1000 timesteps
172  Reward  -82.8115180349
172  Mean  -0.155777524221
172  Std  3.27491455605
-------------------------------
Episode finished after 1000 timesteps
173  Reward  -83.2331700972
173  Mean  -0.155829333163
173  Std  3.27293697664
-------------------------------
Episode finished after 1000 timesteps
174  Reward  -83.4201945625
174  Mean  -0.155928020673
174  Std  3.27102207175
-------------------------------
Episode finished after 1000 timesteps
175  Reward  -84.2692286069
175  Mean  -0.156027428899
175  Std  3.26915305766
-------------------------------
Episode finished after 1000 timesteps
176  Reward  -84.6372402634
176  Mean

-------------------------------
Episode finished after 1000 timesteps
226  Reward  -81.7480377398
226  Mean  -0.159420941235
226  Std  3.1980835955
-------------------------------
Episode finished after 1000 timesteps
227  Reward  -82.5540055453
227  Mean  -0.159503785211
227  Std  3.19852126413
-------------------------------
Episode finished after 1000 timesteps
228  Reward  -82.1186039505
228  Mean  -0.159584510098
228  Std  3.19908984062
-------------------------------
Episode finished after 1000 timesteps
229  Reward  -82.5446948527
229  Mean  -0.159661187127
229  Std  3.19968882943
-------------------------------
Episode finished after 1000 timesteps
230  Reward  -83.4790741366
230  Mean  -0.159799318723
230  Std  3.20037131157
-------------------------------
Episode finished after 1000 timesteps
231  Reward  -83.5070918
231  Mean  -0.159882333252
231  Std  3.20115799675
-------------------------------
Episode finished after 1000 timesteps
232  Reward  -83.960915475
232  Mean  -0

-------------------------------
Episode finished after 1000 timesteps
282  Reward  -83.3691493071
282  Mean  -0.163454095865
282  Std  3.29858714209
-------------------------------
Episode finished after 1000 timesteps
283  Reward  -84.495191888
283  Mean  -0.16359416242
283  Std  3.30264652086
-------------------------------
Episode finished after 1000 timesteps
284  Reward  -85.2013309023
284  Mean  -0.163653200175
284  Std  3.30691459156
-------------------------------
Episode finished after 1000 timesteps
285  Reward  -84.0407988523
285  Mean  -0.163704228326
285  Std  3.31128534099
-------------------------------
Episode finished after 1000 timesteps
286  Reward  -83.1220672968
286  Mean  -0.163814721507
286  Std  3.31557330683
-------------------------------
Episode finished after 1000 timesteps
287  Reward  -83.1553833432
287  Mean  -0.163890856768
287  Std  3.31990460312
-------------------------------
Episode finished after 1000 timesteps
288  Reward  -82.7700838241
288  Mean 

-------------------------------
Episode finished after 1000 timesteps
338  Reward  -84.8076663847
338  Mean  -0.167312463891
338  Std  3.57264973375
-------------------------------
Episode finished after 1000 timesteps
339  Reward  -87.0359239737
339  Mean  -0.167387103835
339  Std  3.58088389697
-------------------------------
Episode finished after 1000 timesteps
340  Reward  -85.4040306465
340  Mean  -0.167468947317
340  Std  3.58949973768
-------------------------------
Episode finished after 1000 timesteps
341  Reward  -84.8485491951
341  Mean  -0.167551863811
341  Std  3.5979488786
-------------------------------
Episode finished after 825 timesteps
342  Reward  28.8599414479
342  Mean  -0.167610001257
342  Std  3.60644460659
-------------------------------
Episode finished after 1000 timesteps
343  Reward  -85.2577750822
343  Mean  -0.167523277332
343  Std  3.59634527222
-------------------------------
Episode finished after 1000 timesteps
344  Reward  -85.6977569884
344  Mean  

-------------------------------
Episode finished after 1000 timesteps
394  Reward  -86.9726276571
394  Mean  -0.170705957193
394  Std  4.07519584687
-------------------------------
Episode finished after 1000 timesteps
395  Reward  -85.9924956738
395  Mean  -0.170765373156
395  Std  4.08802458871
-------------------------------
Episode finished after 1000 timesteps
396  Reward  -87.0246090814
396  Mean  -0.170828959784
396  Std  4.10069214393
-------------------------------
Episode finished after 1000 timesteps
397  Reward  -87.159978651
397  Mean  -0.170908799806
397  Std  4.11362881578
-------------------------------
Episode finished after 1000 timesteps
398  Reward  -87.4581379449
398  Mean  -0.170980344743
398  Std  4.12668488313
-------------------------------
Episode finished after 1000 timesteps
399  Reward  -88.1311486534
399  Mean  -0.171052425476
399  Std  4.1399202659
-------------------------------
Episode finished after 1000 timesteps
400  Reward  -87.4344082032
400  Mean 

-------------------------------
Episode finished after 1000 timesteps
450  Reward  -88.4897175275
450  Mean  -0.173257442774
450  Std  4.76908257289
-------------------------------
Episode finished after 1000 timesteps
451  Reward  -89.5826251114
451  Mean  -0.173323470296
451  Std  4.78595593208
-------------------------------
Episode finished after 1000 timesteps
452  Reward  -89.9517997251
452  Mean  -0.173391781714
452  Std  4.80316123043


# Actor Critic

In [ ]:
value_nn = FerchNN.NN_Network(input_size = env.observation_space.shape[0], lr =0.000001, output_size = 1, activation = "linear", loss = "mse")

In [ ]:
n_episodes = 100000
horizon = 1000
gamma = 0.99
for i_episode in range(n_episodes):
    
    state = env.reset()
    
    actions = []
    means = []
    stds = []
    rewards = []
    time = 0
    for t in range(horizon):
        time+=1
        env.render()
        ########################################
        action, mean, std = choose_action(state)
        actions.append(action)
        means.append(mean)
        stds.append(std)
        ########################################
        next_state, reward, done, info = env.step(action)
        rewards.append(reward)
        
        
        V_next = value_nn.predict(next_state)
        
        if done:
            V_next = np.zeros(1)
        
        TD = reward+ gamma*V_next - value_nn.predict(state)
        
        value_nn.fit(state, reward+ gamma*V_next)
        
        mean_gradient, std_gradient  = compute_gradient(action,TD, mean, std)
        mean_nn.fit(None,None, dloss = mean_gradient)
        std_nn.fit(None, None, dloss = std_gradient)
        
        state = next_state
        if done:
            break
    
    
    print("-------------------------------")
    print("Episode finished after {} timesteps".format(time+1))
    print(i_episode, " Reward ", np.array(rewards).sum())
    print(i_episode, " Mean ", np.array(means).mean())
    print(i_episode, " Std ", std_transformation(np.array(stds)).mean())
    
env.close()